This file is to accaculate the accuracy,Precision,Recall,F1-score of models
At first, ignore the time delay
Accuracy 的判断标准是：
Correct_i = 1, if (label_i=0 and touch_type_idx_i=0)
            1, if (label_i=1 and touch_type_idx_i=GT)
            0, otherwise
​其中：label_map_TCNN = { 0:"NC",1:"ST", 2:"P", 3:"G"}
GT 是 ground truth（真实的手势类别）
"ST" → touch_type_idx == 1
"P" → touch_type_idx == 2
"G" → touch_type_idx == 3
然后计算准确率：
Accuracy=∑𝑖Correct_𝑖/𝑁

考虑延迟的情况：
对于每个 label=1（表示实际发生接触）的时间点 t，在模型预测中寻找时间范围 [t, t + Ts] 内的预测结果。如果在此时间窗口内模型正确识别了手势，则视为一次正确预测。
​
 
​
 


In [11]:
# 不考虑延迟
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 读取 CSV 文件
df = pd.read_csv("../DATA/Labeled_data/0213_2LCNN_Post1_ST/labeled_modelResult_data.csv")

# 提取 Ground Truth (GT) 手势类别
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}
folder_name = "0213_2LCNN_Post1_ST"  # 文件夹名称
gt_label = folder_name.split("_")[-1]  # 提取 "ST", "P", or "G"
gt_idx = gesture_mapping.get(gt_label, -1)  # 获取对应 touch_type_idx

# 计算正确预测的情况
df["correct"] = ((df["label"] == 0) & (df["touch_type_idx"] == 0)) | \
                ((df["label"] == 1) & (df["touch_type_idx"] == gt_idx))

accuracy = df["correct"].mean()
y_true = df["label"].replace({0: 0, 1: gt_idx})  # 0 表示无触碰, 1 表示当前手势类别
y_pred = df["touch_type_idx"]

precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")



# 输出结果

print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")
# df[(df['label'] == 1) & (df['touch_type_idx'] != 1)]


Accuracy: 90.32%
Precision: 33.51%
Recall: 33.64%
F1-score: 33.29%


/Users/shangyu/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
# T秒的延迟
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 读取数据
df = pd.read_csv("../DATA/Labeled_data/0213_2LCNN_Post1_ST/labeled_modelResult_data.csv")

# 提取 Ground Truth (GT) 手势类别
gesture_mapping = {"NC": 0, "ST": 1, "P": 2, "G": 3}
folder_name = "0213_2LCNN_Post1_ST"  # 文件夹名称
gt_label = folder_name.split("_")[-1]  # 提取 "ST", "P", or "G"
gt_idx = gesture_mapping.get(gt_label, -1)  # 获取对应 touch_type_idx

# 定义时间延迟 Ts
Ts = 3  

# 初始化正确预测列表
correct_predictions = []

# 遍历数据集
for i, row in df.iterrows():
    if row["label"] == 1:
        # 查找时间范围 [t, t + Ts] 内的预测
        t = row["time"]
        window = df[(df["time"] >= t) & (df["time"] <= t + Ts)]
        if any(window["touch_type_idx"] == gt_idx):
            correct_predictions.append(1)
        else:
            correct_predictions.append(0)
    else:
        # 对于 label=0 的情况，直接比较
        correct_predictions.append(1 if row["touch_type_idx"] == 0 else 0)

# 将结果添加到数据框
df["correct"] = correct_predictions

# 计算准确率
accuracy = df["correct"].mean()
print(f"Accuracy (considering delay): {accuracy:.2%}")

# 计算 Precision, Recall, F1-score
# 创建新的标签，考虑延迟
y_true = df["label"].replace({0: 0, 1: gt_idx})
y_pred = df["touch_type_idx"]

precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")


Accuracy (considering delay): 93.24%
Precision: 33.51%
Recall: 33.64%
F1-score: 33.29%
